# MiRPipe Flowchart

![title](Figures/miRPipe_synthetic.png)

# FastQ Files
**Loading libraries**

In [ ]:
import os
import csv
import sys
import numpy as np
import pickle
import pandas as pd
import glob
import subprocess
import random
import requests
from splinter import Browser
import time
import networkx as nx
from operator import add
from os import path
import threading
import plotly.graph_objects as go
import subprocess
import tempfile
from rna_tools.Seq import RNASequence
from rna_tools.rna_tools_config import RFAM_DB_PATH
from threading import Semaphore
screenlock = Semaphore(value=1)

In [ ]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

**Declaring ENV Variables**

In [ ]:
# setting env variables:
os.environ['HOME_DIR'] = os.getcwd()
os.environ['SEQ_DIR'] = os.path.join(os.environ['HOME_DIR'],'data')
os.environ['Tools_DIR'] = os.path.join(os.environ['HOME_DIR'],'Tools')    
os.environ['REF_DIR'] = os.path.join(os.environ['HOME_DIR'], 'Tools',
                                     'MDS_command_line_v38/MDS_command_line')
os.mkdir(os.path.join(os.environ['SEQ_DIR'],'output'))
print('Please be sure that all the fastq files are present in %s. All the results will be saved in the same folder also.' %os.environ['SEQ_DIR'])

Ensure that path is correct and fastq files are present in this path
For Smooth Functioning of Pipeline, please make sure that only input fastq files are present in the data folder.
Delete all the clutter before re-running the pipeline for trouble-free execution.

In [ ]:
!echo $HOME_DIR
!echo $SEQ_DIR
!echo $REF_DIR

# Pre-processing
## Quality Check of Fastq Files

In [ ]:
'''
    Following script perform the following task:
    1. Perform quality checking of rach samples using FastQC.
    2. Prepare the better info-graphic and detailed report of all quality checking
       reports from all samples using multiqc
'''
!Rscript $HOME_DIR/scripts/FastQC.R

## Adaptor Timming and Fastq Splitting

In [ ]:
print('The default adaptor sequence used for adaptor trimming is \
TCGTATGCCGTCTTCTGCTTG. If your adaptor sequence is different then please \
enter your data specific adaptor sequence. Please selec appropriate option.')
print('1. Default adaptor sequence: TCGTATGCCGTCTTCTGCTTG')
print('2. User specific adaptor sequence')
user_choice =int(input('Please select your option:'))

if user_choice == 1:
    adaptor = 'TGGAATTCTCGGGTGCCAAGG'
    os.environ['adaptor'] = adaptor
elif user_choice == 2:
    adaptor = input('Please enter the adaptor sequence')
    os.environ['adaptor'] = adaptor    


In [ ]:
# Add script for adaptor trimming here using trim_galore
!bash scripts/adaptor_trimming.sh

# Add script for read length based spliting (into 3 parts) here using bbduk
!bash scripts/fastq_split.sh

# Sequence Alignment 

## **Downloading the Mirdeep* aligner and mirbase**

In [ ]:
print('Please enter your choice: ')
print("Option 1: hg19 based alignment using Mirdeep* with miRBase v19 ")
print("Option 2: hg19 based alignment using Mirdeep* with miRBase v20")
print("Option 3: hg38 based alignment using Mirdeep* with miRBase 21")
print("Option 4: hg38 based alignment using Mirdeep* with miRBase 22 (Default Condition)")
print("Please enter either 1, 2, 3 or 4")
choice = int(input("Please enter your choice:  "))

if choice == 1:
    print("You chose hg19 and miRBase v19 based alignment using Mirdeep*")
    with open('data/mirdeep.conf',"w") as handle:
        handle.write("Human Genome = hg19\nmiRBase = 19")   
    print("Downloading Mirdeep*....")
    command = "wget --content-disposition  http://sourceforge.net/projects/mirdeepstar/files/MDS_command_line_v37.zip -O $Tools_DIR/MDS_command_line_v37.zip && "
    command += "unzip -o $Tools_DIR/MDS_command_line_v37.zip -d $Tools_DIR"
    subprocess.call(command, shell=True)    
    print("Downloading Mirdeep* is complete. Now downloading hg19 human reference genome....")
    ref_var = 'hg19'    
    os.environ['REF_DIR'] = os.path.join(os.environ['HOME_DIR'], 'Tools',
                                     'MDS_command_line_v37/MDS_command_line')
    print("")
    command = ""
    command += "mkdir $HOME_DIR/refs/hg19 && "
    command += "wget http://hgdownload.soe.ucsc.edu/goldenPath/hg19/bigZips/hg19.fa.gz -P $HOME_DIR/refs/hg19/ && "
    command += "pigz -p 5 -d $HOME_DIR/refs/hg19/hg19.fa.gz && echo Download complete. Now building bowtie indexes && "
    command += "$HOME_DIR/Tools/bowtie-1.2.3-linux-x86_64/bowtie-build --threads 8 $HOME_DIR/refs/hg19/hg19.fa $HOME_DIR/refs/hg19/hg19"
    subprocess.call(command, shell=True)  
    print("Human refenrence genome hg19 has been downloaded and index files generation complete.")
    
elif choice == 2:
    print("You chose hg19 and miRBase v20 based alignment using Mirdeep*")
    with open('data/mirdeep.conf',"w") as handle:
        handle.write("Human Genome = hg19\nmiRBase = 20")   
    command = ""
    command = "wget --content-disposition  http://sourceforge.net/projects/mirdeepstar/files/MDS_command_line_v37.zip -O $Tools_DIR/MDS_command_line_v37.zip && "
    command += "unzip -o $Tools_DIR/MDS_command_line_v37.zip -d $Tools_DIR && "
    command += "rm $HOME_DIR/Tools/MDS_command_line_v37/MDS_command_line/genome/hg19/miRBase/* && "
    command += "wget --content-disposition https://www.mirbase.org/ftp/20/hairpin.fa.gz -P $HOME_DIR/Tools/MDS_command_line_v37/MDS_command_line/genome/hg19/miRBase && "
    command += "pigz -p 5 -d $HOME_DIR/Tools/MDS_command_line_v37/MDS_command_line/genome/hg19/miRBase/hairpin.fa.gz && "
    command += "wget --content-disposition  https://www.mirbase.org/ftp/20/mature.fa.gz -P $HOME_DIR/Tools/MDS_command_line_v37/MDS_command_line/genome/hg19/miRBase && "
    command += "pigz -p 5 -d $HOME_DIR/Tools/MDS_command_line_v37/MDS_command_line/genome/hg19/miRBase/mature.fa.gz &&  "
    command += "wget --content-disposition  https://www.mirbase.org/ftp/20/genomes/hsa.gff3 -P $HOME_DIR/Tools/MDS_command_line_v37/MDS_command_line/genome/hg19/miRBase && "
    command += "mv $HOME_DIR/Tools/MDS_command_line_v37/MDS_command_line/genome/hg19/miRBase/hsa.gff3 $HOME_DIR/Tools/MDS_command_line_v37/MDS_command_line/genome/hg19/miRBase/knownMiR.gff3"                
    subprocess.call(command, shell=True)   
    print("Downloading Mirdeep* is complete. Now downloading hg19 human reference genome....")
    ref_var = 'hg19'    
    os.environ['REF_DIR'] = os.path.join(os.environ['HOME_DIR'], 'Tools',
                                     'MDS_command_line_v37/MDS_command_line')
    print("")
    command = ""
    command += "mkdir $HOME_DIR/refs/hg19 && "
    command += "wget http://hgdownload.soe.ucsc.edu/goldenPath/hg19/bigZips/hg19.fa.gz -P $HOME_DIR/refs/hg19 && "
    command += "pigz -p 5 -d $HOME_DIR/refs/hg19/hg19.fa.gz && echo Download complete. Now building bowtie indexes && "
    command += "$HOME_DIR/Tools/bowtie-1.2.3-linux-x86_64/bowtie-build --threads 8 $HOME_DIR/refs/hg19/hg19.fa $HOME_DIR/refs/hg19/hg19"
    subprocess.call(command, shell=True)    
    print("Human refenrence genome hg19 has been downloaded and index files generation complete.")

elif choice == 3:
    print("You chose hg38 and miRBase v21 based genome alignment using Mirdeep*")
    with open('data/mirdeep.conf',"w") as handle:
        handle.write("Human Genome = hg38\nmiRBase = 21")   
    print("Downloading Mirdeep*....")
    command = ""
    command += "rm -rf $Tools_DIR/MDS_command_line_v38/MDS_command_line/genome/hg38 && " 
    command += "wget --content-disposition  http://sourceforge.net/projects/mirdeepstar/files/Index_files/hg38.zip/download -P $Tools_DIR/ && "
    command += "unzip -o $Tools_DIR/hg38.zip -d $Tools_DIR/MDS_command_line_v38/MDS_command_line/genome/ && "
    command += "rm -r $Tools_DIR/MDS_command_line_v38/MDS_command_line/genome/hg19"
    subprocess.call(command, shell=True)   
    print("Downloading Mirdeep* is complete.")
    ref_var = 'hg38'   

elif choice == 4:
    print("You chose hg38 and miRBase v22 based genome alignment using Mirdeep*(Default Condition)")
    with open('data/mirdeep.conf',"w") as handle:
        handle.write("Human Genome = hg38\nmiRBase = 22")   
    ref_var = 'hg38'

else:
    print("Please enter valid option")


## piRNA Pipeline

In [ ]:
def piRNA_pipeline():
    cmd = 'bash scripts/piRNA_pipeline.sh'
    os.system(cmd)

threading.Thread(target=piRNA_pipeline).start()

## miRNA sequence Alignment

Since the synthetic data experiments are performed only on 1 sample that contains reads generated by seed-based simulator miRSim. We don't need multiple threads functionality of miRPipe. So We have uncommented the simple sequential sample processing bash script for sequence alignment using miRDeep* 

In [ ]:
# # Sequential sample processing
!bash $HOME_DIR/scripts/Mirdeep_star.sh

**Preparing batches for miRNA sequence alignment for multi-thread processing**

In [ ]:
# def isTreadAlive():
#     for t in threads:
#         if t.isAlive():
#             return 1
#         return 0

In [ ]:
# def mirdeep_star(batch,batch_id):    
#     os.chdir(os.environ['REF_DIR'])
#     files = os.listdir(os.getcwd())    
#     ref_id = os.path.join(os.getcwd(),'genome')
#     if 'hg19' in os.listdir(ref_id):
#         ref_idx = 'hg19'
#     elif 'hg38' in os.listdir(ref_id):
#         ref_idx = 'hg38'
#     else:
#         print('Reference index are missing in ', ref_id)
#         sys.exit(2)
    
#     batch = pd.read_csv(os.path.join(os.environ['SEQ_DIR'],batch),index_col=0)
#     for idx in range(batch.shape[0]):
#         basename = batch.iloc[idx,0].split('.')[0] +'_trimmed.fastq'
#         path_new = os.path.join(os.environ['SEQ_DIR'],'fastq_17_24',basename)
        
#         # Constraining miRDeep* to take only 4 gb in each thread
#         try:
#             command = 'java -Xmx4096m -jar MD.jar -g '+ ref_idx + ' -a ' + os.environ['adaptor'] + ' -t 17 -l 24 -s -20 -r 5 -p 20 -m 101 ' + path_new + ' && rm ' + path_new
#             os.system(command)
#         except:
#             screenlock.acquire()
#             print('Sample ',basename,' is not present in the directory.')
#             screenlock.release()
    
#     screenlock.acquire()
#     print('---------------------------')
#     print(batch_id, ' is complete.')
#     print('---------------------------')
#     screenlock.release()

In [ ]:
# Collecting the sample_list provided by user
# os.chdir(os.environ['HOME_DIR'])
# sample_list = pd.read_csv('data/sample_list.csv',index_col=0)

# # Generating batches
# if sample_list.shape[0] < 10:
#     no_of_batch = 4
# elif sample_list.shape[0] > 10 and sample_list.shape[0] <= 30:
#     no_of_batch = 8
# elif sample_list.shape[0] > 30 and sample_list.shape[0] <= 50:
#     no_of_batch = 12
# elif sample_list.shape[0] > 50 and sample_list.shape[0] <= 100:
#     no_of_batch = 16
# elif sample_list.shape[0] > 100:
#     no_of_batch = 20
    
# file_list_splitted = np.array_split(sample_list, no_of_batch)
# for batch_idx in range(len(file_list_splitted)):
#     file_list_splitted[batch_idx].to_csv('data/batch_'+str(batch_idx+1)+'.csv',encoding='utf-8',index=True)

In [ ]:
# Calling mirdeep* for each batch in individual threads.
# threads = []
# for i in range(1,no_of_batch+1):
#     print('Running batch_'+ str(i)+ ' on thread-'+ str(i))
#     t = threading.Thread(target=mirdeep_star,args=('batch_'+str(i)+'.csv', 'batch_'+str(i),))
#     threads.append(t)
#     t.start()

# flag =1
# while (flag):
#     time.sleep(5.0)
#     flag = isTreadAlive()

# Post-Alignment Analysis
## **Processing of raw counts from Mirdeep* results**

In [ ]:
os.chdir(os.getcwd())
files = [] 
[files.append(i) for i in os.listdir("data/fastq_17_24") if (".result" in i and not "known" in i)]
result_data = {}
print('Preparing master look up table for result data')
for file in tqdm(files):    
    result_data['_'.join(file.split("_")[:-1])] = {}
    readfile = open(os.path.join("data/fastq_17_24", file), 'r').readlines()
    header = readfile[0].split("\t")
    for line in readfile[1:]:
        result_data['_'.join(file.split("_")[:-1])][line.split("\t")[0]] = {}
        result_data['_'.join(file.split("_")[:-1])][line.split("\t")[0]][header[1]] = line.split("\t")[1]
        result_data['_'.join(file.split("_")[:-1])][line.split("\t")[0]][header[2]] = line.split("\t")[2]
        result_data['_'.join(file.split("_")[:-1])][line.split("\t")[0]][header[3]] = line.split("\t")[3]
        result_data['_'.join(file.split("_")[:-1])][line.split("\t")[0]][header[4]] = line.split("\t")[4]
        result_data['_'.join(file.split("_")[:-1])][line.split("\t")[0]][header[5]] = int(line.split("\t")[5])
        result_data['_'.join(file.split("_")[:-1])][line.split("\t")[0]][header[6]] = line.split("\t")[6]
        result_data['_'.join(file.split("_")[:-1])][line.split("\t")[0]][header[7]] = line.split("\t")[7]
        result_data['_'.join(file.split("_")[:-1])][line.split("\t")[0]][header[8]] = line.split("\t")[8]
        result_data['_'.join(file.split("_")[:-1])][line.split("\t")[0]][header[9]] = line.split("\t")[9]
        result_data['_'.join(file.split("_")[:-1])][line.split("\t")[0]][header[10]] = line.split("\t")[10]

    known_miR_filename = file.split('.')[0] + '.known_miR.result'
    readfile = open(os.path.join("data/fastq_17_24", known_miR_filename), 'r').readlines()
    for line in readfile[1:]:
        if not line.split("\t")[0] in result_data['_'.join(file.split("_")[:-1])].keys():
            result_data['_'.join(file.split("_")[:-1])][line.split("\t")[0]] = {}
            result_data['_'.join(file.split("_")[:-1])][line.split("\t")[0]][header[1]] = line.split("\t")[1]
            result_data['_'.join(file.split("_")[:-1])][line.split("\t")[0]][header[2]] = line.split("\t")[2]
            result_data['_'.join(file.split("_")[:-1])][line.split("\t")[0]][header[3]] = line.split("\t")[3]
            result_data['_'.join(file.split("_")[:-1])][line.split("\t")[0]][header[4]] = line.split("\t")[4]
            result_data['_'.join(file.split("_")[:-1])][line.split("\t")[0]][header[5]] = int(line.split("\t")[5])
            result_data['_'.join(file.split("_")[:-1])][line.split("\t")[0]][header[6]] = line.split("\t")[6]
            result_data['_'.join(file.split("_")[:-1])][line.split("\t")[0]][header[7]] = line.split("\t")[7]
            result_data['_'.join(file.split("_")[:-1])][line.split("\t")[0]][header[8]] = line.split("\t")[8]
            result_data['_'.join(file.split("_")[:-1])][line.split("\t")[0]][header[9]] = line.split("\t")[9].split('(')[0].lower()
            result_data['_'.join(file.split("_")[:-1])][line.split("\t")[0]][header[10]] = line.split("\t")[10]
        else:
            result_data['_'.join(file.split("_")[:-1])][line.split("\t")[0]][header[5]] += int(line.split("\t")[5])


        
print('Master lookup table is generated. Now collecting all unique miRs from master lookup table')
mir_dict = {}
for file in tqdm(result_data.keys()):
    # print('Working on ',file)
    for k,v in result_data[file].items():
        i = 1
        if not k in mir_dict.keys():
            mir_dict[k] = [v['chr'],v['mature_loci'].split('-')[0],
                           v['mature_loci'].split('-')[1],v['mature miR'],v['sequence']]
        else:
            mir_dict1 = mir_dict.copy()
            for k1,v1 in mir_dict1.items():
                new_loc_start = int(v['mature_loci'].split('-')[0])
                new_loc_end = int(v['mature_loci'].split('-')[0])
                new_mir_seq = v['mature miR']
                if k1 == k:                    
                    if v1[0] == v['chr']:                          
                        if (new_loc_start == int(v1[1])) and (new_loc_end == int(v1[2])):
                            if new_mir_seq == v1[3]:
                                pass
                    else:
                        mir_dict[k+'_'+str(i)] = [v['chr'],v['mature_loci'].split('-')[0],
                                       v['mature_loci'].split('-')[1],v['mature miR'],v['sequence']]
                        i += 1


print('Prepating dataframe for all unique miRs and samples....')
counts = pd.DataFrame(columns=list(result_data.keys()))
mir_name = list(mir_dict.keys())
index = 0

for k,v in tqdm(mir_dict.items()):
    for file in result_data.keys():
        for k1,v1 in result_data[file].items():
            if v1['mature miR'] == v[3]:
                loc_start = int(v1['mature_loci'].split('-')[0])
                loc_end = int(v1['mature_loci'].split('-')[1])
                loc_chr = v1['chr']
                if loc_chr == v[0]:
                    if loc_start == int(v[1]) and loc_end == int(v[2]):   
                        counts.loc[k,file] = int(v1['expression(number of mature reads)'])
                   
counts['mir_ID'] = mir_name
counts = counts.set_index('mir_ID')
counts['chr'] = [v[0] for v in mir_dict.values()]
counts['chr_start'] = [v[1] for v in mir_dict.values()]
counts['chr_end'] = [v[2] for v in mir_dict.values()]
counts['miRNA_sequence'] = [v[3] for v in mir_dict.values()]
counts['miRNA_precursor_sequence'] = [v[4].lower() for v in mir_dict.values()]
# rearranging columns
cols = counts.columns.tolist()
cols = cols[-5:]+ cols[:-5]
counts = counts[cols]

counts = counts.fillna(0)
print(counts.shape)
counts.to_csv("data/counts.csv",encoding='utf-8',index=True)
counts.head()

In [ ]:
counts_known = counts[counts.index.str.contains('hsa')]
counts_unknown = counts[~counts.index.str.contains('hsa')]
print('All %d known and %d novel miRNA are successfully separated' %(counts_known.shape[0],counts_unknown.shape[0]))

# Blast Search
## Search for Novel miRNA sequence in DashR Database

In [ ]:
'''
   Run this cell only when running this notebook first time. After successfully running this, 
   pickle file will be saved which can be used further without generating the 
   results again and will save lot of time.
   This code has been tested with Firefox version 66.0.4 (64-bit).
'''

novel_mirs = list(counts_unknown.index.values)
gecko = os.path.abspath('geckodriver')
dashR_Results = []
browser = Browser('firefox',executable_path=gecko,headless=True)
browser.visit('http://dashr2.lisanwanglab.org/search.php#')

if ref_var == 'hg19':
    #  Select DASHR2 GEO HG19 as reference
    xpath = '/html/body/div/div[1]/div/div/select/option[3]' 
else:
    #  Select DASHR2 GEO HG38 as reference
    xpath = '/html/body/div/div[1]/div/div/select/option[4]'
browser.find_by_xpath(xpath).click()
time.sleep(3)

browser.find_by_text('Search by sequence ').click()
time.sleep(3)
n_idx = len(list(counts_unknown.loc[:,'miRNA_sequence']))
n = 0
for idx in tqdm(range(n_idx)):
    name = list(counts_unknown.index.values)[idx]
    seq = list(counts_unknown.loc[:,'miRNA_sequence'])[idx]
    dashR_Results.append(name)
    browser.fill('querySeq', seq)
    time.sleep(2)
    xpath = '//*[@id="search"]/div[4]/div/button'    
    browser.find_by_xpath(xpath).click()
    if not browser.is_text_present('No matches found'):
        n += 1
        xpath = '//*[@id="sequence-results"]/pre/table' 
        results = browser.find_by_xpath(xpath)
        i=0
        for search_result in results:
            title = search_result.text.encode('utf8') 
            link = search_result["href"]             
            dashR_Results.append((title, link)) 
            i += 1   
    else:
        seq = str(Seq(seq).reverse_complement())
        browser.fill('querySeq', seq)
        time.sleep(2)
        xpath = '//*[@id="search"]/div[4]/div/button'    
        browser.find_by_xpath(xpath).click()
        if not browser.is_text_present('No matches found'):
            n += 1
            xpath = '//*[@id="sequence-results"]/pre/table' 
            results = browser.find_by_xpath(xpath)
            i=0
            for search_result in results:
                title = search_result.text.encode('utf8') 
                link = search_result["href"]             
                dashR_Results.append((title, link)) 
                i += 1
        else:
            dashR_Results.append('miRNA NOT FOUND')
    


browser.quit()
with open("data/dashR_Results.pickle", 'wb') as handle:
    pickle.dump(dashR_Results, handle, protocol=pickle.HIGHEST_PROTOCOL)

Uncomment the following cell if you want to load the pickle file obtained from dashR search module.

In [ ]:
# '''
#    Get DashR database searching results from saved pickle file.   
# '''
# import pickle
# print('Loading saved results from DashR Database....')
# with open("data/dashR_Results.pickle", 'rb') as handle:
#     dashR_Results = pickle.load(handle)
# len(dashR_Results)

## DashR Results post-processing

In [ ]:
def loc_check(rna_name, df1,idx):
    
    rna_name1 = rna_name.split(' ')[1]
    if '3p' in rna_name1 or '5p' in rna_name1:
        rna_name1 = rna_name1[:-3]
    rna_name1 = rna_name1.lower()
    
    rna_chr1 = counts_unknown.loc[dashR_Results[idx],'chr']
    rna_chr_start1 = int(counts_unknown.loc[dashR_Results[idx],'chr_start'])
    rna_chr_end1 = int(counts_unknown.loc[dashR_Results[idx],'chr_end'])

    if not counts_unknown.shape[0] == df1.shape[0]:
        try:
            rna_chr2 = df1.loc[rna_name1,'chr']
            rna_chr_start2 = int(df1.loc[rna_name1,'chr_start'])
            rna_chr_end2 = int(df1.loc[rna_name1,'chr_end'])
        except:
            rna_chr2 = df1.loc[dashR_Results[idx],'chr']
            rna_chr_start2 = int(df1.loc[dashR_Results[idx],'chr_start'])
            rna_chr_end2 = int(df1.loc[dashR_Results[idx],'chr_end'])

    else:
        rna_chr2 = rna_name.split(' ')[3].split(':')[0]
        rna_chr_start2 = int(rna_name.split(' ')[3].split(':')[1].split('[')[0].split('-')[0])
        rna_chr_end2 = int(rna_name.split(' ')[3].split(':')[1].split('[')[0].split('-')[1])
    
    if (rna_chr1 == rna_chr2) and np.abs(rna_chr_start1 - rna_chr_start2)<=2 and np.abs(rna_chr_end1 - rna_chr_end2)<=2:
        flag = True
        return flag

In [ ]:
counts_unknown2 = counts_unknown.copy()
counts_known2 = counts_known.copy()
counts_unknown2 = counts_unknown2.drop(['miRNA_sequence','chr','chr_start','chr_end','miRNA_precursor_sequence'],axis=1)
counts_known2 = counts_known2.drop(['miRNA_sequence','chr','chr_start','chr_end','miRNA_precursor_sequence'],axis=1)
novel_mir_old_index = counts_unknown.index.tolist()
paralogue_dict = {}

for idx in (range(0,len(dashR_Results),2)):
    results = dashR_Results[idx+1]    
    if isinstance((dashR_Results[idx+1]),tuple):
        results = list(dashR_Results[idx+1])[0].decode("utf-8")        
        rna_type_results = results.split('\n')[-1]
        rna_type1 = rna_type_results.split(' ')[1].lower()
        if 'hsa-mir' in rna_type1:                            
            idx2 = list(counts_unknown2.index.values).index(dashR_Results[idx])
            count_to_add = list(map(int,list(counts_unknown2.iloc[idx2,:].values)))
            if '3p' in rna_type1 or '5p' in rna_type1:
                rna_type1 = rna_type1[:-3]
            if rna_type1.lower() in list(counts_known.index.values):    
                counts_add_flag = loc_check(rna_type_results,counts_known,idx)
                if counts_add_flag:                        
                    count_increament = list(counts_known2.loc[rna_type1.lower(),:].values)                        
                    count_increament = list(map(add,count_increament,count_to_add))               
                    counts_known2.iloc[list(counts_known2.index.values).index(rna_type1.lower()),:] = count_increament
                    counts_unknown2 = counts_unknown2.drop([str(dashR_Results[idx])],axis = 0)                    
                    novel_mir_old_index.remove(novel_mir_old_index[novel_mir_old_index.index(dashR_Results[idx])])
                else:                    
                    if rna_type1 in paralogue_dict.keys():
                        char_string = paralogue_dict[rna_type1][-1].split('_')[-1]
                        char_no = int(char_string) + 1
                    else:
                        char_no = 1

                    new_name = rna_type1 + '_' + str(char_no)
                    unique_flag = False
                    while unique_flag == False:
                        if new_name in counts_known2.index.tolist():
                            new_name = new_name[:-2]+'_' + str(int(new_name[-1])+1)
                        else:
                            unique_flag = True

                    old_index = counts_unknown2.index.tolist()                        
                    old_index[idx2] = new_name
                    counts_unknown2.index = old_index
                    if rna_type1 in paralogue_dict.keys():
                        paralogue_dict[rna_type1].append(new_name)
                    else:
                        paralogue_dict[rna_type1] = []
                        paralogue_dict[rna_type1].append(new_name)
            else:
                mir_add_flag = loc_check(rna_type_results,counts_unknown,idx)
                if mir_add_flag:
                    char_no += 1
                    # Novel mir name will be changed to known mir name
                    if old_index[idx2] in counts_known2.index.tolist():
                        old_index[idx2] = old_index[idx2][:-2] + '_'+ str(char_no)
                        char_no += 1
                    old_index = counts_unknown2.index.tolist()
                    old_index[idx2] = rna_type1[:-2] + '_'+ str(char_no)
                    counts_unknown2.index = old_index
                else:
                    # Novel miRNA is the paralogues of known mirna
                    if rna_type1 in paralogue_dict.keys():
                        char_string = paralogue_dict[rna_type1][-1].split('_')[-1]
                        char_no = int(char_string) + 1
                    else:
                        char_no = 1

                    new_name = rna_type1 + '_' + str(char_no)
                    unique_flag = False
                    while unique_flag == False:
                        if new_name in counts_known2.index.tolist():
                            new_name = new_name[:-2]+'_' + str(int(new_name[-1])+1)
                        else:
                            unique_flag = True
                    old_index = counts_unknown2.index.tolist()                        
                    old_index[idx2] = new_name
                    counts_unknown2.index = old_index 
                    if rna_type1 in paralogue_dict.keys():
                        paralogue_dict[rna_type1].append(new_name)
                    else:
                        paralogue_dict[rna_type1] = []
                        paralogue_dict[rna_type1].append(new_name)
            

counts_known2['chr'] = counts_known[["chr"]]
counts_known2['chr_start'] = counts_known[["chr_start"]]
counts_known2['chr_end'] = counts_known[["chr_end"]]
counts_known2['miRNA_sequence'] = counts_known[["miRNA_sequence"]]
counts_known2['miRNA_precursor_sequence'] = counts_known[["miRNA_precursor_sequence"]]

new_mir_list = counts_unknown2.index.tolist()
counts_unknown2.index = novel_mir_old_index
counts_unknown2['chr'] = counts_unknown[["chr"]]
counts_unknown2['chr_start'] = counts_unknown[["chr_start"]]
counts_unknown2['chr_end'] = counts_unknown[["chr_end"]]
counts_unknown2['miRNA_sequence'] = counts_unknown[["miRNA_sequence"]]
counts_unknown2['miRNA_precursor_sequence'] = counts_unknown[["miRNA_precursor_sequence"]]
counts_unknown2.index = new_mir_list

# rearranging columns
cols = counts_known2.columns.tolist()
cols = cols[-5:]+ cols[:-5]
counts_known2 = counts_known2[cols]

cols = counts_unknown2.columns.tolist()
cols = cols[-5:]+ cols[:-5]
counts_unknown2 = counts_unknown2[cols]

# add 2 dfs
new_mirs_df = counts_unknown2[counts_unknown2.index.str.contains('hsa')]
frames = [counts_known2, new_mirs_df]
counts_known2 = pd.concat(frames)
counts_unknown2 = counts_unknown2[~counts_unknown2.index.str.contains('hsa')]

In [ ]:
print(counts_unknown2.shape)
print(counts_known2.shape)

# Seed based Sequence Clustering
## **Dictionary Preparation**

In [ ]:
counts_known2 = counts_known2.drop_duplicates(subset=['miRNA_sequence'], keep='first')
seed_dict = {}
Xseed_dict = {}
unknown_mir_dict = {}
for index in counts_unknown2.index:
    seed = counts_unknown2.loc[index,'miRNA_sequence'][1:7]
    Xseed = counts_unknown2.loc[index,'miRNA_sequence'][7:]
    unknown_mir_dict[index] = [seed,counts_unknown2.loc[index,'chr'],counts_unknown2.loc[index,'chr_start'],
                              counts_unknown2.loc[index,'chr_end'],counts_unknown2.loc[index,'miRNA_sequence']]
    seed_dict[index] = seed
    Xseed_dict[index] = Xseed

for index in counts_known2.index:
    seed = counts_known2.loc[index,'miRNA_sequence'][1:7]
    Xseed = counts_known2.loc[index,'miRNA_sequence'][7:]
    unknown_mir_dict[index] = [seed,counts_known2.loc[index,'chr'],counts_known2.loc[index,'chr_start'],
                              counts_known2.loc[index,'chr_end'],counts_known2.loc[index,'miRNA_sequence']]
    seed_dict[index] = seed
    Xseed_dict[index] = Xseed
    

In [ ]:
from collections import defaultdict 

exact_seed_cluster = defaultdict(list) 
for key,values in seed_dict.items():
    exact_seed_cluster[values].append(key)
    
exact_seed_cluster = dict(exact_seed_cluster)

exact_Xseed_cluster = defaultdict(list) 
for key,values in Xseed_dict.items():
    exact_Xseed_cluster[values].append(key)
    
exact_Xseed_cluster = dict(exact_Xseed_cluster)

**Fasta File**

In [ ]:
count = 1
writer = open("data/seed_seqs.fasta", 'w')
for key in unknown_mir_dict.keys():
    line = ">" + str(count) + "\t" + key + "\n"
    line += str(unknown_mir_dict[key][0]) + "\n"
    writer.write(line)
    count+=1
writer.close()

In [ ]:
count = 1
writer = open("data/seed_outer_seqs.fasta", 'w')
for key in unknown_mir_dict.keys():
    line = ">" + str(count) + "\t" + key + "\n"
    line += str(unknown_mir_dict[key][-1][7:]) + "\n"
    writer.write(line)
    count+=1
writer.close()

## CD-HIT

In [ ]:
# Execute CD-HIT command

!cd Tools/cd-hit-v4.6.8-2017-1208/; echo "***** Clustering miRs Seed Sequences Only**********"; \
                                                           ./cd-hit -l 5 -n 2 -c 1 \
                                                                    -i ../../data/seed_seqs.fasta \
                                                                    -o ../../data/cdhit_seed;     \
                                                           echo " "; \
                                                           echo "***** Clustering miRs Rest Sequences Only**********"; \
                                                           ./cd-hit -l 8 -i ../../data/seed_outer_seqs.fasta \
                                                                    -o ../../data/cdhit_seed_outer;

## Seed Clustering Algorithm

Preparing lookup table for Xseed Clusters

In [ ]:
"""
CLuster dictionary preparation
Cluster : cluster_no:[mir1,mir2]
"""
def fetch_id(xseed_id_fasta, xid):
    for line in xseed_id_fasta:
        if line:
            if '>' in line[0]:   
                if xid == line.split('\t')[0].split('>')[1]:
                    return line.split('\t')[1]


"""

"""

def cluster_dict_preparation(cluster_file, fasta_file):
    cluster_file = open(cluster_file,'r').read().split('\n')
    fasta_id = open(fasta_file,'r').read().split('\n')
    cluster_dict = {}    
    for line in cluster_file:
        if line:
            if '>' in line[0]:
                line_no = cluster_file.index(line)
                flag = True    
                k = line[1:].replace(' ','_')
                xid = cluster_file[line_no+1].split('>')[1].split('.')[0]
                cluster_dict[k] = [fetch_id(fasta_id,xid)]
                i = 2
                while flag:
                    if cluster_file[line_no+i]:
                        if '>' in cluster_file[line_no+i][0]:
                            flag = False
                            pass
                        else:
                            xid = cluster_file[line_no+i].split('>')[1].split('.')[0]
                            cluster_dict[k].append(fetch_id(fasta_id,xid))
                            i += 1
                    else:
                        flag = False
                        
    return cluster_dict


seed_cluster = cluster_dict_preparation('data/cdhit_seed.clstr','data/seed_seqs.fasta')
Xseed_cluster = cluster_dict_preparation('data/cdhit_seed_outer.clstr','data/seed_outer_seqs.fasta')

In [ ]:
"""
Only for outer seed region clusters
rev_dict : mir_name:[cluster1,cluster2]
"""


def rev_dictionary(cluster):
    rev_dict = defaultdict(list)
    mir_list = []
    for k in list(unknown_mir_dict.keys()):
        mir_list.append(k)
        
    for mir in mir_list:
        for k,v in cluster.items():
            for mir1 in v:
                if mir == mir1:
                    rev_dict[mir].append(k)
                    
    return dict(rev_dict)


exact_seed_rev_dict = rev_dictionary(exact_seed_cluster)
seed_rev_dict = rev_dictionary(seed_cluster)
Xseed_rev_dict = rev_dictionary(Xseed_cluster)

In [ ]:
"""
Function to update and merge the counts if all conditions (seed,seq and loc) are satisfied
Input : df1,df2,mir1 (from df1), mir2 (from df2)
Output : df1_modified (with updated counts of mir1) and df2_modified (with mir2 removed)
"""

def update_counts(df1,df2,mir1,mir2):
    mir1_counts = list(df1.loc[mir1,:].values)[5:]
    mir2_counts = list(df2.loc[mir2,:].values)[5:]
    mir2_counts = list(map(add,mir1_counts,mir2_counts))
    df1.loc[mir2,:] = list(df1.loc[mir2,:].values)[:5] + mir2_counts
    df2 = df2.drop(mir1,axis = 0)
    return df1, df2

In [ ]:
novel_miRNA2 = counts_unknown2.copy()
mir_reannotation = pd.DataFrame()
mir_old_name = []
mir_new_name = []
novel_mir_seq = []
family_name = []
attribute = []
paralogue_dict = {}

for k,v in tqdm(seed_cluster.items()):    
    char_no = 0
    novel_mir_name = seed_cluster[k]
    novel_mir_name = [i for i in novel_mir_name if not 'hsa' in i]
    for n_mir in novel_mir_name:   
        novel_mir_name = [i for i in novel_mir_name if i is not None]                        
        if len(novel_mir_name)>1:  
            for n_mir in novel_mir_name:
                if not n_mir == None:
                    char_no = 1
                    novel_mir_name1 = novel_mir_name.copy()
                    novel_mir_name1.remove(n_mir)
                    n_chr_loc = unknown_mir_dict[n_mir][1]
                    n_chr_loc_start = int(unknown_mir_dict[n_mir][2])
                    n_chr_loc_end = int(unknown_mir_dict[n_mir][3])
                    seed_common_mir = list(set(novel_mir_name1) & set(exact_seed_cluster[exact_seed_rev_dict[n_mir][0]]))
                    if seed_common_mir:                    
                        Xseed_common_mir = list(set(seed_common_mir) & set(Xseed_cluster[Xseed_rev_dict[n_mir][0]]))
                        if Xseed_common_mir:
                            for c_mir1 in Xseed_common_mir:
                                c_chr_loc = unknown_mir_dict[c_mir1][1]
                                c_chr_loc_start = int(unknown_mir_dict[c_mir1][2])
                                c_chr_loc_end = int(unknown_mir_dict[c_mir1][3])
                                if (n_chr_loc == c_chr_loc) and np.abs(n_chr_loc_start - c_chr_loc_start)<=2 and np.abs(n_chr_loc_end - c_chr_loc_end)<=2:
                                    _,novel_miRNA2 = update_counts(novel_miRNA2,novel_miRNA2,c_mir1,n_mir)
                                    novel_mir_name[novel_mir_name.index(c_mir1)] = None
                                    mir_old_name.append(c_mir1)
                                    mir_new_name.append(n_mir)
                                    family_name.append(n_mir)
                                    novel_mir_seq.append(unknown_mir_dict[n_mir][-1])
                                    attribute.append('Same Seed, Xseed and loc')
                                else:
                                    if n_mir in paralogue_dict.keys():
                                        char_string = paralogue_dict[n_mir][-1].split('_')[-1]
                                        char_no = int(char_string) + 1
                                    else:
                                        char_no = 1
                                        
                                    new_name = n_mir+'_'+ str(char_no)
                                    unique_flag = False
                                    while unique_flag == False:
                                        if new_name in novel_miRNA2.index.tolist():
                                            new_name = new_name[:-2]+'_' + str(int(new_name[-1])+1)
                                        else:
                                            unique_flag = True
                                    novel_miRNA2 = novel_miRNA2.rename(index={c_mir1 : new_name})
                                    """
                                        The incoming miR shared same seed, altered xseed with different genomic location with cluster head.
                                        We need to check whether the incoming miR shares the genomic location with the existing 
                                        members of the same cluster or not. If not, then it'll be added to the cluster with 
                                        different name. Otherwise, it'll be merged to the member with whom it shares the exact 
                                        seed, xseed and genomic location.
                                    """
                                    
                                    if n_mir in paralogue_dict.keys():
                                        if len(paralogue_dict[n_mir]) >= 1:
                                            for m in paralogue_dict[n_mir]:
                                                if m in novel_miRNA2.index.tolist():
                                                    if (novel_miRNA2.loc[m,'chr'] == c_chr_loc) and np.abs(int(novel_miRNA2.loc[m,'chr_start']) - int(novel_miRNA2.loc[new_name,'chr_start']))<=2 and np.abs(int(novel_miRNA2.loc[m,'chr_end']) - int(novel_miRNA2.loc[new_name,'chr_end']))<=2:
                                                        _,novel_miRNA2 = update_counts(novel_miRNA2,novel_miRNA2,new_name,m)
                                                        mir_old_name.append(c_mir1)
                                                        mir_new_name.append(m)
                                                        family_name.append(m)
                                                        novel_mir_seq.append(unknown_mir_dict[n_mir][-1])
                                                        attribute.append('Same Seed, Xseed and loc')
                                                        break
                                    novel_mir_name[novel_mir_name.index(c_mir1)] = None
                                    mir_old_name.append(c_mir1)
                                    mir_new_name.append(new_name)
                                    family_name.append(n_mir)
                                    novel_mir_seq.append(unknown_mir_dict[n_mir][-1])
                                    attribute.append('Paralogues')                                    
                                    if n_mir in paralogue_dict.keys():
                                        paralogue_dict[n_mir].append(new_name)
                                    else:
                                        paralogue_dict[n_mir] = []
                                        paralogue_dict[n_mir].append(new_name)
                        else:
                            for c_mir1 in seed_common_mir: 
                                c_chr_loc = unknown_mir_dict[c_mir1][1]
                                c_chr_loc_start = int(unknown_mir_dict[c_mir1][2])
                                c_chr_loc_end = int(unknown_mir_dict[c_mir1][3])                               
                                if n_mir in paralogue_dict.keys():
                                        char_string = paralogue_dict[n_mir][-1].split('_')[-1]
                                        char_no = int(char_string) + 1
                                else:
                                    char_no = 1
                                    
                                new_name = n_mir+'_'+ str(char_no)
                                unique_flag = False
                                while unique_flag == False:
                                    if new_name in novel_miRNA2.index.tolist():
                                        new_name = new_name[:-2]+'_' + str(int(new_name[-1])+1)
                                    else:
                                        unique_flag = True
                                novel_miRNA2 = novel_miRNA2.rename(index={c_mir1 : new_name})
                                if n_mir in paralogue_dict.keys():
                                        if len(paralogue_dict[n_mir]) >= 1:
                                            for m in paralogue_dict[n_mir]:
                                                if m in novel_miRNA2.index.tolist():
                                                    if (novel_miRNA2.loc[m,'chr'] == c_chr_loc) and np.abs(int(novel_miRNA2.loc[m,'chr_start']) - int(novel_miRNA2.loc[new_name,'chr_start']))<=2 and np.abs(int(novel_miRNA2.loc[m,'chr_end']) - int(novel_miRNA2.loc[new_name,'chr_end']))<=2:
                                                        _,novel_miRNA2 = update_counts(novel_miRNA2,novel_miRNA2,new_name,m)
                                                        mir_old_name.append(c_mir1)
                                                        mir_new_name.append(m)
                                                        family_name.append(m)
                                                        novel_mir_seq.append(unknown_mir_dict[n_mir][-1])
                                                        attribute.append('Same Seed, Xseed and loc')
                                                        break
                                novel_mir_name[novel_mir_name.index(c_mir1)] = None
                                mir_old_name.append(c_mir1)
                                mir_new_name.append(new_name)
                                family_name.append(n_mir)
                                novel_mir_seq.append(unknown_mir_dict[n_mir][-1])
                                attribute.append('Paralogues')                                    
                                if n_mir in paralogue_dict.keys():
                                    paralogue_dict[n_mir].append(new_name)
                                else:
                                    paralogue_dict[n_mir] = []
                                    paralogue_dict[n_mir].append(new_name)

                                

            
            
mir_reannotation['Old_miR_ID'] = mir_old_name
mir_reannotation['New_miR_ID'] = mir_new_name
mir_reannotation['Family'] = family_name
mir_reannotation['Relation'] = attribute
mir_reannotation['sequence'] = novel_mir_seq
mir_reannotation.head()

In [ ]:
seed_dict2 = {k:v for k,v in seed_dict.items() if 'hsa' in k}
for idx in tqdm(range(novel_miRNA2.shape[0])):
    novel_seq = novel_miRNA2.iloc[idx,3]
    novel_seed = novel_seq[1:7]
    suffix = 1
    for k,v in seed_dict2.items():
        if novel_seed == v:
            if k+'_'+str(suffix) in novel_miRNA2.index:
                paralog_index = novel_miRNA2[novel_miRNA2.index.str.contains(k)].index.tolist()
                suffix = sorted([int(i.split('_')[-1]) for i in paralog_index])[-1]
                new_name = k + '_' + str(suffix+1)
            elif not '_' in k:
                suffix = 1
                new_name = k + '_' + str(suffix)
            novel_miRNA2 = novel_miRNA2.rename(index={novel_miRNA2.index[idx] : new_name})
            break    

In [ ]:
frame = [counts_known2,novel_miRNA2]
counts_final = pd.concat(frame)
counts_final = counts_final.drop_duplicates(subset=['miRNA_sequence'], keep='first')
counts_final.to_csv("data/counts_final.csv",encoding='utf-8',index=True)
print(counts_final.shape)

## Pipeline evalutation

In [ ]:
mirna_ground_truth = pd.read_csv('data/mirna2.csv')
novelrna_ground_truth = pd.read_csv('data/novel_mirna2.csv')
pirna_ground_truth = pd.read_csv('data/pirna2.csv')
frame = [mirna_ground_truth,novelrna_ground_truth,pirna_ground_truth]
ground_truth = pd.concat(frame)
novelrna_ground_truth.head()

In [ ]:
def rna_to_dna(seq):
    for char in seq:
        seq = list(seq)
        char_idx = seq.index(char)
        if char == 'u':
            seq[char_idx] = 't'
            
    seq1 = ""
    return seq1.join(seq).upper()

In [ ]:
pirna_counts = pd.read_csv('data/piRNA/pirna_counts/synthetic_data_trimmed_pirna_counts.txt',sep='\t')
pirna_counts.columns = ['chr','database','type','chr_start','chr_end','Sequence','strand','extra','RNA_ID','Depth']
pirna_counts =pirna_counts[pirna_counts['Depth']!=0]
pirna_counts = pirna_counts[['RNA_ID','Sequence','Depth']]
pirna_counts1 = pirna_counts[['RNA_ID','Depth']]
pirna_counts1 = pirna_counts1.set_index('RNA_ID')
pirna_counts = pirna_counts.set_index('RNA_ID')
pirna_counts.to_csv('data/piRNA/pirna_counts/pirna_counts.csv',encoding='utf-8',index=True)
# print(pirna_counts.shape)
pirna_counts2 = pd.DataFrame(pirna_counts1.groupby(pirna_counts1.index)['Depth'].sum())
# print(pirna_counts2.shape)

mirna_count = pd.read_csv('data/counts_final.csv',index_col=0)
mir_final1 = pd.DataFrame()
mir_final1['RNA_ID'] = list(mirna_count.index)
mir_final1['Sequence'] = [rna_to_dna(seq) for seq in mirna_count['miRNA_sequence']]
mir_final1['Depth'] = list(mirna_count['synthetic_data_trimmed.result'])
mir_final1 = mir_final1.set_index('RNA_ID')
frame = [pirna_counts2,mir_final1]
result = pd.concat(frame,sort=True)
# result = pd.read_csv('data/counts_final.csv',index_col=0)
result.head()

In [ ]:
seq_dict = {}
for idx in tqdm(range(ground_truth.shape[0])):
    seq = ground_truth.iloc[idx,4]
    impurity = ground_truth.iloc[idx,1]
    name = ground_truth.iloc[idx,0]
    counts = int(ground_truth.iloc[idx,-1])
    chr_loc = ground_truth.iloc[idx,5]
    if seq in seq_dict.keys():
        if seq_dict[seq][2] == impurity:
            seq_dict[seq][-2] += counts
    else:
        seq_dict[seq] = [name,seq,impurity,counts,chr_loc]

In [ ]:
mir_final2 = pd.DataFrame()
actual_mir_name = []
actual_count = []
impurity = []
# pir_list = [pir for pir in list(mir_ground_truth_final['RNA_ID']) if 'hsa-piR' in pir]
seq_list = list(set(ground_truth['ref_Sequence']))
for idx in tqdm(range(result.shape[0])):
    seq = result.iloc[idx,1]
    if str(seq) == 'nan':
        for k,v in seq_dict.items():
            if result.index[idx] == v[0] and v[-3] == 'None':
#                 print(str(idx), k,v[0],pirna_counts2.loc[list(result.index)[idx],:])
                mir_final2 = mir_final2.append(pirna_counts2.loc[list(result.index)[idx],:])
                actual_mir_name.append(result.index[idx])
                actual_count.append(v[-2])
                impurity.append(v[-3])
    else:
        for seq1 in seq_list:
            if seq1 in seq or seq in seq1:        
                actual_mir_name.append(seq_dict[seq1][0])
                actual_count.append(int(seq_dict[seq1][-2]))
                impurity.append(seq_dict[seq1][-3])
                if result.loc[[list(result.index)[idx]],:].shape[0] == 1:
                    mir_final2 = mir_final2.append(result.loc[list(result.index)[idx],:])
                else:
                    df = result.loc[list(result.index)[idx],:]
                    df2 = df.iloc[[0],:]                    
                    df2['Depth'] = df['Depth'].sum()
                    mir_final2 = mir_final2.append(df2.loc[df2.index[0],:])
            else:
                pass


mir_final2['actual_name'] = actual_mir_name
mir_final2['actual_counts'] = actual_count
mir_final2['impurity'] = impurity
mir_final2.to_csv(os.path.join('data/mirs_matched_with_ground_truth.csv'),encoding='utf-8',index=True)
print(mir_final2.shape)
mir_final2.head()

In [ ]:
df_actual_kmir_none = mirna_ground_truth[mirna_ground_truth['Impure_Region'].str.contains('None')]
df_actual_kmir_seed = mirna_ground_truth[mirna_ground_truth['Impure_Region'].str.contains('^Seed_region')]
df_actual_kmir_xseed = mirna_ground_truth[mirna_ground_truth['Impure_Region'].str.contains('Outside_Seed_region')]
df_actual_kmir_both = mirna_ground_truth[mirna_ground_truth['Impure_Region'].str.contains('Both_region')]
positive_counts_kmir = df_actual_kmir_none['Expression_count'].sum()
negative_counts_kmir = df_actual_kmir_seed['Expression_count'].sum() + df_actual_kmir_xseed['Expression_count'].sum() + df_actual_kmir_both['Expression_count'].sum()

df_actual_nmir_none = novelrna_ground_truth[novelrna_ground_truth['Impure_Region'].str.contains('None')]
df_actual_nmir_seed = novelrna_ground_truth[novelrna_ground_truth['Impure_Region'].str.contains('^Seed_region')]
df_actual_nmir_xseed = novelrna_ground_truth[novelrna_ground_truth['Impure_Region'].str.contains('Outside_Seed_region')]
df_actual_nmir_both = novelrna_ground_truth[novelrna_ground_truth['Impure_Region'].str.contains('Both_region')]
positive_counts_nmir = df_actual_nmir_none['Expression_count'].sum()
negative_counts_nmir = df_actual_nmir_seed['Expression_count'].sum() + df_actual_nmir_xseed['Expression_count'].sum() + df_actual_nmir_both['Expression_count'].sum()

df_actual_pmir_none = pirna_ground_truth[pirna_ground_truth['Impure_Region'].str.contains('None')]
df_actual_pmir_seed = pirna_ground_truth[pirna_ground_truth['Impure_Region'].str.contains('^Seed_region')]
df_actual_pmir_xseed = pirna_ground_truth[pirna_ground_truth['Impure_Region'].str.contains('Outside_Seed_region')]
df_actual_pmir_both = pirna_ground_truth[pirna_ground_truth['Impure_Region'].str.contains('Both_region')]
positive_counts_pmir = df_actual_pmir_none['Expression_count'].sum()
negative_counts_pmir = df_actual_pmir_seed['Expression_count'].sum() + df_actual_pmir_xseed['Expression_count'].sum() + df_actual_pmir_both['Expression_count'].sum()

total_positive = positive_counts_kmir + positive_counts_nmir + positive_counts_pmir
total_negative = negative_counts_kmir + negative_counts_nmir + negative_counts_pmir

In [ ]:
mir_final = pd.read_csv('data/mirs_matched_with_ground_truth.csv',index_col=0)
df0 = mir_final[(mir_final['actual_name'].str.contains('hsa-miR')) |\
                (mir_final['actual_name'].str.contains('hsa-mir')) |\
               (mir_final['actual_name'].str.contains('hsa-let'))] 
df0 = df0.sort_values(by ="Depth", ascending=False)
df0.drop_duplicates(subset ="Sequence", keep = 'first', inplace = True)
for idx in range(df0.shape[0]):
    if df0.iloc[idx,0] > df0.iloc[idx,3]:
        df0.iloc[idx,0] = df0.iloc[idx,3]

df = df0[(df0.index.str.contains('hsa-mir')) |\
         (df0.index.str.contains('hsa-miR')) |\
        (df0.index.str.contains('hsa-let'))]
df1_k = df[df['impurity']=='None']
df1_1k = df[df['impurity']!='None']
df1 = df0[df0.index.str.contains('novel')]
df2 = df0[df0.index.str.contains('hsa-piR')]    
k_k = df1_k['Depth'].sum()
if k_k > positive_counts_kmir:
    print('k_k is equal to positive_counts_kmir')
    k_k = positive_counts_kmir

k_n = df1['Depth'].sum()
k_p = df2['Depth'].sum()
k_o = positive_counts_kmir - k_k - k_n - k_p
c_matrix = pd.DataFrame(index = ["known_mirna","novel_mirna","pirna","other"], columns=["known_mirna","novel_mirna","pirna","other"])
c_matrix.iloc[0,0] = k_k
c_matrix.iloc[1,0] = k_n
c_matrix.iloc[2,0] = k_p
c_matrix.iloc[3,0] = k_o

# For novel miRNA    
df0 = mir_final[((mir_final['actual_name'].str.contains('novel')))]
df0 = df0.sort_values(by=['Depth'], ascending=False)
df0.drop_duplicates(subset ="Sequence", keep = 'first', inplace = True)
for idx in range(df0.shape[0]):
    if df0.iloc[idx,0] > df0.iloc[idx,3]:
        df0.iloc[idx,0] = df0.iloc[idx,3]
df1 = df0[df0.index.str.contains('novel')]
df1_n = df1[df1['impurity']=='None']
df1_1n = df1[df1['impurity']!='None']
df2 = df0[df0.index.str.contains('hsa-mir') | df0.index.str.contains('hsa-miR')]
df3 = df0[df0.index.str.contains('hsa-piR')]        
n_k = df2['Depth'].sum()
n_n = df1_n['Depth'].sum()
if n_n > positive_counts_nmir:
    print('n_n is equal to positive_counts_nmir')
    n_n = positive_counts_nmir

n_p = df3['Depth'].sum()
n_o = positive_counts_nmir - n_k - n_n - n_p
c_matrix.iloc[0,1] = n_k
c_matrix.iloc[1,1] = n_n
c_matrix.iloc[2,1] = n_p
c_matrix.iloc[3,1] = n_o

# For piRNA
df0 = mir_final[((mir_final['actual_name'].str.contains('hsa-piR')))]
for idx in range(df0.shape[0]):
    if df0.iloc[idx,0] > df0.iloc[idx,3]:
        df0.iloc[idx,0] = df0.iloc[idx,3]
df1 = df0[df0.index.str.contains('hsa-mir') | df0.index.str.contains('hsa-miR')]
df2 = df0[df0.index.str.contains('novel')]
df3 = df0[df0.index.str.contains('piR')]
df1_p = df3[df3['impurity']=='None']
df1_1p = df3[df3['impurity']!='None']
p_k = df1['Depth'].sum()
p_n = df2['Depth'].sum()
p_p = df1_p['Depth'].sum()
p_o = positive_counts_pmir - p_k - p_n - p_p
c_matrix.iloc[0,2] = p_k
c_matrix.iloc[1,2] = p_n
c_matrix.iloc[2,2] = p_p
c_matrix.iloc[3,2] = p_o

# For remaining reads
o_k = df1_1k['Depth'].sum()
o_n = df1_1n['Depth'].sum()
o_p = df1_1p['Depth'].sum()
o_o = total_negative - o_k - o_n - o_p

c_matrix.iloc[0,3] = o_k
c_matrix.iloc[1,3] = o_n
c_matrix.iloc[2,3] = o_p
c_matrix.iloc[3,3] = o_o

c_matrix.to_csv("data/multi_class_c_matrix.csv",encoding="utf-8",index=True)

cm = c_matrix.to_numpy()
c = c_matrix
# for known miRNA
tp_k = c.iloc[0,0]
fn_k = c['known_mirna'][1:].sum()
fp_k = c.iloc[0,1:].sum()
tn_k = np.trace(c) - tp_k
acc = 100*(tp_k+tn_k)/(tp_k+tn_k+fp_k+fn_k)
prec = 100*tp_k/(tp_k+fp_k)
sen = 100*tp_k/(tp_k+fn_k)
spec = 100*tn_k/(fp_k+tn_k)
f1_sc = 100*2*tp_k/(2*tp_k+fp_k+fn_k)
k_mir_param = [acc,prec,sen,spec,f1_sc]
k_mir_param = list(map(float, k_mir_param))

# for novel miRNA
tp_n = c.iloc[1,1]

fn_n = c.iloc[0,1] + c['novel_mirna'][2:].sum()
fp_n = c.iloc[1,0] + c.iloc[1,2:].sum()
tn_n = np.trace(c) - tp_n
if tp_n == 0:
    acc = 0
    prec = 0
    sen = 0
    spec = 0
    f1_sc = 0
else:
    acc = 100*(tp_n+tn_n)/(tp_n+tn_n+fp_n+fn_n)
    prec = 100*tp_n/(tp_n+fp_n)
    sen = 100*tp_n/(tp_n+fn_n)
    spec = 100*tn_n/(fp_n+tn_n)
    f1_sc = 100*2*tp_n/(2*tp_n+fp_n+fn_n)
n_mir_param = [acc,prec,sen,spec,f1_sc]
n_mir_param = list(map(float, n_mir_param))

# for piRNA
tp_p = c.iloc[2,2]
fn_p = c['pirna'][:2].sum() + c.iloc[3,2]
fp_p = c.iloc[2,3] + c.iloc[2,:2].sum()
tn_p = np.trace(c) - tp_p
if tp_p == 0:
    acc = 0
    prec = 0
    sen = 0
    spec = 0
    f1_sc = 0
else:
    acc = 100*(tp_p+tn_p)/(tp_p+tn_p+fp_p+fn_p)
    prec = 100*tp_p/(tp_p+fp_p)
    sen = 100*tp_p/(tp_p+fn_p)
    spec = 100*tn_p/(fp_p+tn_p)
    f1_sc = 100*2*tp_p/(2*tp_p+fp_p+fn_p)
p_mir_param = [acc,prec,sen,spec,f1_sc]
p_mir_param = list(map(float, p_mir_param))

# for all RNA
acc = 100*np.trace(c)/c.to_numpy().sum()
prec = np.mean([k_mir_param[1],n_mir_param[1],p_mir_param[1]])
sen = np.mean([k_mir_param[2],n_mir_param[2],p_mir_param[2]])
spec = np.mean([k_mir_param[3],n_mir_param[3],p_mir_param[3]])
f1_sc = np.mean([k_mir_param[4],n_mir_param[4],p_mir_param[4]])
all_mir_param = [acc,prec,sen,spec,f1_sc]

out_f = pd.DataFrame()
out_f['param'] = ['accuracy', 'precision', 'sensitivity', 'specificity', 'f1_score']
out_f['known_mirna'] = k_mir_param
out_f['novel_mirna'] = n_mir_param
out_f['pirna'] = p_mir_param
out_f['overall'] = all_mir_param
out_f = out_f.set_index('param')
out_f.to_csv("data/multi_classc_matrix_performance.csv",encoding='utf-8',index=True)

In [ ]:
pd.read_csv('data/multi_class_c_matrix.csv',index_col=0)

In [ ]:
pd.read_csv('data/multi_classc_matrix_performance.csv',index_col=0)

# Output Files

Two output files will be generated after completing the synthetic data experiment using this notebook. These files are as follows:

    1. multi_class_c_matrix.csv: This file contains all the sequence counts for each category of RNA.
    
    2. multi_classc_matrix_performance.csv: This file contains all the performance matrices for miRPipe pipeline.